In [ ]:
!pip install sagemaker -U

In [ ]:
import sagemaker
import boto3
sess=sagemaker.Session()

# create a bucket to store data,model files,etc
sagemaker_bucket=None
if sagemaker_bucket is None and sess is not None:
    sagemaker_bucket=sess.default_bucket()

# Role Management
try:
    role=sagemaker.get_execution_role()
except ValueError:
    iam=boto3.client("iam")
    role=iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

session_sagemaker=sagemaker.Session(default_bucket=sagemaker_bucket)
print(f"sagemaker role ARN: {role}")
print(f"sagemaker session region:{sess.boto_region_name}")

In [6]:
from sagemaker.huggingface.model import HuggingFaceModel

# Hub configuration
hub={
    'HF_MODEL_ID':"distilbert-base-uncase-distilled-squad",
    'HF_TASK':"question-answering"
}

# create HF Model class
hf_model=HuggingFaceModel(
    env=hub,
    role=role,
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version='py39'
)

In [ ]:
# Depoly it to sagemaker inference
predictor=hf_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge"
)

# Input and context
data={
"inputs":{
    "question":"What is used as an instance?",
    "context":"My name is Akash, I live in Mumbai. This model is used with sagemaker. The model is using ml.m5.xlarge instance to run."
}
}

# request
predictor.predict(data)

In [ ]:
predictor.predict(data)